In [38]:
import os
import pandas as pd

os.chdir('../../')

## lmsys-chat-1m

In [39]:
df = pd.read_parquet('data/lmsys-chat-1m/train-00000-of-00006-4feeb3f83346a0e9.parquet')

df.head(10)

,conversation_id,model,conversation,turn,language,openai_moderation,redacted
0,33f01939a744455c869cb234afca47f1,wizardlm-13b,[{'content': 'how can identity protection serv...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
1,1e230e55efea4edab95db9cb87f6a9cb,vicuna-13b,[{'content': 'Beside OFAC's selective sanction...,6,English,"[{'categories': {'harassment': False, 'harassm...",False
2,0f623736051f4a48a506fd5933563cfd,vicuna-13b,[{'content': 'You are the text completion mode...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
3,e5c923a7fa3f4893beb432b4a06ef222,palm-2,[{'content': 'The sum of the perimeters of thr...,2,English,"[{'categories': {'harassment': False, 'harassm...",False
4,8ad66650dced4b728de1d14bb04657c1,vicuna-13b,[{'content': 'What is the type of the variable...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
5,aa041ed88edd4100bde61b8d68fc7288,wizardlm-13b,[{'content': 'I have 1000 documents to downloa...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
6,113d3ddd85874229a04a660bc629c2cc,vicuna-13b,"[{'content': 'summarise below transcript ""Stud...",1,English,"[{'categories': {'harassment': False, 'harassm...",False
7,4c95520511844ca492ad9ec1cb3672e3,llama-2-13b-chat,[{'content': 'Определи важнейшие смыслы в текс...,3,unknown,"[{'categories': {'harassment': False, 'harassm...",False
8,64f322dcb69d43229bbd9785b7d90f1b,vicuna-13b,"[{'content': 'Buenas noches!', 'role': 'user'}...",8,Spanish,"[{'categories': {'harassment': False, 'harassm...",False
9,6fc9a36392e94a83939dc3738ab9e245,vicuna-13b,[{'content': 'hola puedes hablar español de ar...,5,Spanish,"[{'categories': {'harassment': False, 'harassm...",False


### Возьмем только англоязычные сообщения

In [40]:
df.value_counts(df['language'])

language
English       129664
Portuguese      4821
Russian         4705
unknown         4585
Chinese         4121
               ...  
Tibetan            1
Tigrinya           1
Swati              1
Tamil              1
Yiddish            1
Name: count, Length: 142, dtype: int64

In [41]:
df = df[df['language'] == 'English']

df.head(10)

,conversation_id,model,conversation,turn,language,openai_moderation,redacted
0,33f01939a744455c869cb234afca47f1,wizardlm-13b,[{'content': 'how can identity protection serv...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
1,1e230e55efea4edab95db9cb87f6a9cb,vicuna-13b,[{'content': 'Beside OFAC's selective sanction...,6,English,"[{'categories': {'harassment': False, 'harassm...",False
2,0f623736051f4a48a506fd5933563cfd,vicuna-13b,[{'content': 'You are the text completion mode...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
3,e5c923a7fa3f4893beb432b4a06ef222,palm-2,[{'content': 'The sum of the perimeters of thr...,2,English,"[{'categories': {'harassment': False, 'harassm...",False
4,8ad66650dced4b728de1d14bb04657c1,vicuna-13b,[{'content': 'What is the type of the variable...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
5,aa041ed88edd4100bde61b8d68fc7288,wizardlm-13b,[{'content': 'I have 1000 documents to downloa...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
6,113d3ddd85874229a04a660bc629c2cc,vicuna-13b,"[{'content': 'summarise below transcript ""Stud...",1,English,"[{'categories': {'harassment': False, 'harassm...",False
10,2d9631f925044e47abcc147e64a0268e,vicuna-13b,[{'content': 'Please focus on preparing for th...,1,English,"[{'categories': {'harassment': False, 'harassm...",True
11,7f4abf64593c439f8b085585aeeda566,koala-13b,[{'content': 'Give me an introduction over 200...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
13,e3addcd33c9d42b2be07c4bbbf9ce92e,llama-2-13b-chat,[{'content': 'how many floors does the burj al...,6,English,"[{'categories': {'harassment': False, 'harassm...",True


### Уберем ненужные колонки

In [42]:
df.drop(['conversation_id', 'model', 'openai_moderation', 'redacted', 'language', 'turn'], inplace=True, axis=1)
df.head(10)

,conversation
0,[{'content': 'how can identity protection serv...
1,[{'content': 'Beside OFAC's selective sanction...
2,[{'content': 'You are the text completion mode...
3,[{'content': 'The sum of the perimeters of thr...
4,[{'content': 'What is the type of the variable...
5,[{'content': 'I have 1000 documents to downloa...
6,"[{'content': 'summarise below transcript ""Stud..."
10,[{'content': 'Please focus on preparing for th...
11,[{'content': 'Give me an introduction over 200...
13,[{'content': 'how many floors does the burj al...


### Сбор нового датасета по пользователям

In [43]:
new_rows = []

for user_id, convo in enumerate(df['conversation']):
    previous_model_message = None
    for msg in convo:
        if msg['role'] == 'user':
            new_rows.append({
                'user_id': user_id,
                'user_message': msg['content'],
                'previous_model_message': previous_model_message
            })
        elif msg['role'] == 'assistant':
            previous_model_message = msg['content']

new_df = pd.DataFrame(new_rows)
new_df.to_csv('data/lmsys-chat-1m/processed/dialogs.csv', index=False)
new_df.head(10)

,user_id,user_message,previous_model_message
0,0,how can identity protection services help prot...,None
1,1,Beside OFAC's selective sanction that target t...,None
2,1,are there other types of US sanctions that you...,The United States has a number of different ty...
3,1,please make organized conclusion in bullet lis...,"Yes, there are several other types of US sanct..."
4,1,"can you please revise the answer above again, ...",* Selective sanctions: targeted at specific in...
5,1,if you see a person name stating that it is th...,"Sure, here is a revised list of the different ..."
6,1,if you see a person's name stating that it is ...,
7,2,You are the text completion model and you must...,None
8,3,The sum of the perimeters of three equal squar...,None
9,3,ty,The perimeter of one square is 36 / 3 = 12 cm....


### Проверка на полноту и целостность

In [44]:
df = pd.read_csv("data/lmsys-chat-1m/processed/dialogs.csv")

print("Пропущенные значения:")
print(df.isnull().sum())

print(f"Уникальных пользователей: {df['user_id'].nunique()}")

empty_user_messages = df[df['user_message'].str.strip().eq('') | df['user_message'].isnull()]
print(f"Пустых или отсутствующих user_message: {len(empty_user_messages)}")

no_previous_model_msg = df['previous_model_message'].isnull().sum()
print(f"Сообщений без previous_model_message (начало диалога): {no_previous_model_msg}")

user_message_count = df['user_message'].notnull().sum()
print(f"Всего сообщений от пользователя: {user_message_count}")

model_message_count = df['previous_model_message'].notnull().sum()
print(f"Всего сообщений от модели (предыдущих): {model_message_count}")

Пропущенные значения:
user_id                        0
user_message                   0
previous_model_message    130831
dtype: int64
Уникальных пользователей: 129664
Пустых или отсутствующих user_message: 16
Сообщений без previous_model_message (начало диалога): 130831
Всего сообщений от пользователя: 250310
Всего сообщений от модели (предыдущих): 119479


### Выбор и извлечение признаков

**Цель:** Создать информативные признаки, которые могут описать поведение пользователей и эмоциональные оттенки (паралингвистические признаки и другие).

**Категории признаков:**

#### А) Паралингвистические признаки:

- **Использование пунктуации:**
  - Частота восклицательных и вопросительных знаков.
  - Многоточия («...»).

- **Использование эмодзи и символов:**
  - Признаки наличия/отсутствия и количества эмодзи.

- **Длина:**
  - Длина сообщений (количество слов).

- **Стилистические особенности:**
  - Использование заглавных букв (например, сообщения капсом).
  - Повтор слов и переформулировки.
  - Орфографические ошибки (их количество и регулярность).

#### Б) Лингвистические и семантические признаки:

- Сентимент-анализ (позитив, негатив, нейтральность).

#### В) Диалоговые признаки (контекст взаимодействия):

- Количество взаимодействий до завершения диалога.
- Переключение тем или возвращение к ранее обсуждаемой теме (анализ расстояния между векторами сообщений).

### Подсчет количества слов в сообщениях

In [45]:
def count_words(text):
    if pd.isnull(text):
        return 0
    return len(text.strip().split())

df['word_count'] = df['user_message'].apply(count_words)

df_total_words = (
    df.groupby('user_id')['word_count']
      .sum()
      .reset_index()
      .rename(columns={'word_count': 'total_words'})
)

df_total_words.head(10)


,user_id,total_words
0,0,11
1,1,150
2,2,66
3,3,29
4,4,18
5,5,46
6,6,182
7,7,111
8,8,25
9,9,59


### Подсчет количества восклицательных и вопросительных знаков

In [46]:
df['exclamation_count'] = df['user_message'].fillna('').apply(lambda x: x.count('!'))

df['question_count'] = df['user_message'].fillna('').apply(lambda x: x.count('?'))

df_punct_totals = df.groupby('user_id')[['exclamation_count', 'question_count']].sum().reset_index()
df_punct_totals = df_punct_totals.rename(columns={
    'exclamation_count': 'total_exclamations',
    'question_count': 'total_questions'
})

df_punct_totals.head(10)

,user_id,total_exclamations,total_questions
0,0,0,0
1,1,0,4
2,2,0,0
3,3,0,0
4,4,0,0
5,5,0,2
6,6,0,4
7,7,0,3
8,8,0,0
9,9,0,0


### Подсчет многоточий (.. и ...)

In [47]:
import re

def count_ellipsis(text):
    if pd.isnull(text):
        return 0
    # Находим все вхождения из двух или более точек подряд
    return len(re.findall(r'\.{2,}', text))

df['ellipsis_count'] = df['user_message'].apply(count_ellipsis)

df_ellipsis = df.groupby('user_id')['ellipsis_count'].sum().reset_index()
df_ellipsis = df_ellipsis.rename(columns={'ellipsis_count': 'total_ellipses'})

df_ellipsis.head(10)


,user_id,total_ellipses
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,0
9,9,0


### Подсчет использования эмодзи

In [48]:
# pip install emoji

import emoji
import re

# типичные текстовые смайлики
text_smileys = [
    r":\)", r":D", r";\)", r":\(", r":P", r":3", r":O", r":'\(", r":'\)", r":-D", r"o_O", r"O_o",
    r":-P", r":-/", r":-\\", r":\|", r":-X", r"<3", r":-S", r":-@", r":\^\)", r"\^_\^", r"-_-", r"XD"
]

smileys_pattern = re.compile("|".join(text_smileys))

# Функция для подсчёта всех эмодзи и смайликов
def count_all_faces(text):
    if pd.isnull(text):
        return 0
    emoji_count = sum(1 for char in text if char in emoji.EMOJI_DATA)
    smiley_count = len(smileys_pattern.findall(text))
    return emoji_count + smiley_count

df['paraling_face_count'] = df['user_message'].apply(count_all_faces)

df_faces = df.groupby('user_id')['paraling_face_count'].sum().reset_index()
df_faces = df_faces.rename(columns={'paraling_face_count': 'total_paraling_faces'})

df_faces.head(10)


,user_id,total_paraling_faces
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,0
9,9,0


### Подсчет слов, которые были полностью написаны капсом

In [49]:
# подсчёт слов, написанных полностью капсом
def count_all_caps_words(text):
    if pd.isnull(text):
        return 0
    # Делим на слова
    return sum(1 for word in text.split() if word.isalpha() and word.isupper() and len(word) > 1)

df['caps_word_count'] = df['user_message'].apply(count_all_caps_words)

df_caps = df.groupby('user_id')['caps_word_count'].sum().reset_index()
df_caps = df_caps.rename(columns={'caps_word_count': 'total_caps_words'})

df_caps.head(10)

,user_id,total_caps_words
0,0,0
1,1,1
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,1
9,9,0


### Подсчет повтора слов

In [50]:
from collections import Counter
import re

def count_repeated_words(text):
    if pd.isnull(text):
        return 0
    words = re.findall(r'\b\w+\b', text.lower())
    word_counts = Counter(words)
    repeated = [word for word, count in word_counts.items() if count > 1]
    return len(repeated)

df['repeated_word_count'] = df['user_message'].apply(count_repeated_words)

df_repeats = df.groupby('user_id')['repeated_word_count'].sum().reset_index()
df_repeats = df_repeats.rename(columns={'repeated_word_count': 'total_repeated_words'})

df_repeats.head(10)


,user_id,total_repeated_words
0,0,1
1,1,18
2,2,16
3,3,3
4,4,2
5,5,6
6,6,38
7,7,14
8,8,2
9,9,4


### Подсчет переформулировок

In [51]:
import torch

from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Используется устройство: {device}")

# Загружаем предобученную модель для эмбеддингов предложений
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',  device=device)

# подсчёт переформулировок у одного пользователя
def count_reformulations(messages):
    messages = [msg for msg in messages if isinstance(msg, str) and msg.strip()]
    if len(messages) < 2:
        return 0
    # Получаем эмбеддинги сообщений
    embeddings = model.encode(messages, convert_to_tensor=False, device=device)
    sims = cosine_similarity(embeddings[:-1], embeddings[1:])
    # Считаем количество "похожих, но не идентичных" сообщений
    count = sum(1 for sim in sims.diagonal() if 0.8 <= sim < 0.99) # s < 0.8 - непохожие, s > 0.99 - копи паста
    return count

results = []
for user_id, group in tqdm(df.groupby('user_id'), desc="Processing users"):
    messages = group['user_message'].tolist()
    reform_count = count_reformulations(messages)
    results.append({'user_id': user_id, 'total_reformulations': reform_count})

df_reform = pd.DataFrame(results)

df_reform.head(10)

Используется устройство: cuda


Processing users: 100%|██████████| 129664/129664 [06:22<00:00, 339.20it/s]


,user_id,total_reformulations
0,0,0
1,1,3
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,0
9,9,0


### Подсчет орфографических ошибок

In [52]:
# pip install pyspellchecker

from spellchecker import SpellChecker
import re

spell = SpellChecker()

# подсчёт орфографических ошибок в одном сообщении
def count_spelling_errors(text):
    if pd.isnull(text):
        return 0
    # Приводим к нижнему регистру и выделяем слова
    words = re.findall(r'\b[a-zA-Z]{2,}\b', text.lower())
    misspelled = spell.unknown(words)
    return len(misspelled)

df['spelling_error_count'] = df['user_message'].apply(count_spelling_errors)

df_spell_errors = df.groupby('user_id')['spelling_error_count'].sum().reset_index()
df_spell_errors = df_spell_errors.rename(columns={'spelling_error_count': 'total_spelling_errors'})

df_spell_errors.head(10)


,user_id,total_spelling_errors
0,0,0
1,1,3
2,2,0
3,3,0
4,4,3
5,5,0
6,6,3
7,7,0
8,8,2
9,9,3


### Сентиментальный анализ

In [53]:
# pip install vaderSentiment

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def classify_sentiment(text):
    if pd.isnull(text) or not text.strip():
        return 'neutral'  # пустые = нейтральные
    scores = analyzer.polarity_scores(text)
    compound = scores['compound']
    if compound >= 0.05:
        return 'positive'
    elif compound <= -0.05:
        return 'negative'
    else:
        return 'neutral'

df['sentiment'] = df['user_message'].apply(classify_sentiment)

df_sentiment_counts = (
    df.groupby(['user_id', 'sentiment'])
      .size()
      .unstack(fill_value=0)
      .reset_index()
)

df_sentiment_counts = df_sentiment_counts.rename(columns={
    'positive': 'positive_count',
    'negative': 'negative_count',
    'neutral': 'neutral_count'
})

df_sentiment_counts.columns.name = None
df_sentiment_counts.head(10)


,user_id,negative_count,neutral_count,positive_count
0,0,0,0,1
1,1,1,2,3
2,2,0,0,1
3,3,0,1,1
4,4,0,1,0
5,5,0,0,1
6,6,0,0,1
7,7,0,0,1
8,8,1,0,0
9,9,0,6,0


### Количество взаимодействий (сообщений пользователя)

In [54]:
df_message_counts = (
    df.groupby('user_id')
      .size()
      .reset_index(name='total_user_messages')
)

df_message_counts.head(10)

,user_id,total_user_messages
0,0,1
1,1,6
2,2,1
3,3,2
4,4,1
5,5,1
6,6,1
7,7,1
8,8,1
9,9,6


### Сбор финального датасета feature

In [55]:
# Словарь всех признаков по user_id
feature_dfs = {
    'total_words': df_total_words,                     # общее число слов в диалоге
    'punctuation': df_punct_totals,                    # ! и ? всего
    'ellipses': df_ellipsis,                           # многоточия
    'emojis_smileys': df_faces,                        # эмодзи + смайлики
    'caps_words': df_caps,                             # капс-слова
    'word_repeats': df_repeats,                        # повторы слов
    'reformulations': df_reform,                       # переформулировки
    'spelling_errors': df_spell_errors,                # орфографические ошибки
    'sentiment': df_sentiment_counts,                  # позитив / негатив / нейтрально
    'messages_count': df_message_counts,               # кол-во взаимодействий
}

# признаки, которые включаем в итоговый датасет
include = [
    'total_words',
    'punctuation',
    'ellipses',
    'emojis_smileys',
    'caps_words',
    'word_repeats',
    'reformulations',
    'spelling_errors',
    'sentiment',
    'messages_count'
]

df_features = None
df_len = None
for name, df_part in feature_dfs.items():
    if name in include:
        if df_features is None:
            df_features = df_part.copy()
            df_len = len(df_part)
        else:
            assert df_len == len(df_part), f'{name} dataset has a different number of rows compared to the others'
            df_features = df_features.merge(df_part, on='user_id', how='left')

df_features.to_csv('data/lmsys-chat-1m/processed/features.csv', index=False)
df_features.head(10)

,user_id,total_words,total_exclamations,total_questions,total_ellipses,total_paraling_faces,total_caps_words,total_repeated_words,total_reformulations,total_spelling_errors,negative_count,neutral_count,positive_count,total_user_messages
0,0,11,0,0,0,0,0,1,0,0,0,0,1,1
1,1,150,0,4,0,0,1,18,3,3,1,2,3,6
2,2,66,0,0,0,0,0,16,0,0,0,0,1,1
3,3,29,0,0,0,0,0,3,0,0,0,1,1,2
4,4,18,0,0,0,0,0,2,0,3,0,1,0,1
5,5,46,0,2,0,0,0,6,0,0,0,0,1,1
6,6,182,0,4,0,0,0,38,0,3,0,0,1,1
7,7,111,0,3,0,0,0,14,0,0,0,0,1,1
8,8,25,0,0,0,0,1,2,0,2,1,0,0,1
9,9,59,0,0,0,0,0,4,0,3,0,6,0,6


### Создадим нормализованный по количеству сообщений датасет

In [56]:
    # Копируем только нужные числовые колонки (кроме user_id и total_messages)
columns_to_normalize = [
    col for col in df_features.columns
    if col not in ['user_id', 'total_user_messages']
    and pd.api.types.is_numeric_dtype(df_features[col])
]

# Нормализуем, сохраняя оригинальные названия колонок
df_normalized = df_features[['user_id']].copy()
for col in columns_to_normalize:
    df_normalized[col] = df_features[col] / df_features['total_user_messages']

df_normalized.to_csv('data/lmsys-chat-1m/processed/features_normalized.csv', index=False)
df_normalized.head(10)

,user_id,total_words,total_exclamations,total_questions,total_ellipses,total_paraling_faces,total_caps_words,total_repeated_words,total_reformulations,total_spelling_errors,negative_count,neutral_count,positive_count
0,0,11.000000,0.0,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.0,0.000000,0.000000,1.0
1,1,25.000000,0.0,0.666667,0.0,0.0,0.166667,3.000000,0.5,0.5,0.166667,0.333333,0.5
2,2,66.000000,0.0,0.000000,0.0,0.0,0.000000,16.000000,0.0,0.0,0.000000,0.000000,1.0
3,3,14.500000,0.0,0.000000,0.0,0.0,0.000000,1.500000,0.0,0.0,0.000000,0.500000,0.5
4,4,18.000000,0.0,0.000000,0.0,0.0,0.000000,2.000000,0.0,3.0,0.000000,1.000000,0.0
5,5,46.000000,0.0,2.000000,0.0,0.0,0.000000,6.000000,0.0,0.0,0.000000,0.000000,1.0
6,6,182.000000,0.0,4.000000,0.0,0.0,0.000000,38.000000,0.0,3.0,0.000000,0.000000,1.0
7,7,111.000000,0.0,3.000000,0.0,0.0,0.000000,14.000000,0.0,0.0,0.000000,0.000000,1.0
8,8,25.000000,0.0,0.000000,0.0,0.0,1.000000,2.000000,0.0,2.0,1.000000,0.000000,0.0
9,9,9.833333,0.0,0.000000,0.0,0.0,0.000000,0.666667,0.0,0.5,0.000000,1.000000,0.0
